In [ ]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db
    
#pip install ipython-sql comes installed in the jupyter lab, we only need to download sqlalchemy.
#This is package tested thoroughly with SQLite, MySQL and Postgre databases.

In [ ]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

--Here we check our schema

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [ ]:
%%sql 

WITH customer_total_purchases AS
    (
     SELECT 
        c.customer_id customer_id,
        SUM(i.total) total_purchases,
        c.support_rep_id support_rep_id
        FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
        GROUP BY 1
    )

SELECT 
    e.employee_id employee_id,
    e.first_name || " " ||e.last_name full_name,
    SUM(ctp.total_purchases) total_purchases,
    ROUND (CAST (SUM(ctp.total_purchases) AS FLOAT)/( 
                                              SELECT SUM(total) FROM invoice  
                                             ), 3)  percent 
   
FROM employee e 
INNER JOIN customer_total_purchases ctp ON e.employee_id = ctp.support_rep_id
GROUP BY 1
ORDER BY 3 DESC

/*
All employees perform equally  good 1/3 of the total 
ledger produced. 
*/

 * sqlite:///chinook.db
Done.


employee_id,full_name,total_purchases,percent
3,Jane Peacock,1731.5099999999998,0.368
4,Margaret Park,1584.0000000000002,0.336
5,Steve Johnson,1393.92,0.296


In [ ]:
%%sql

WITH
    count_inv_USA AS 
    (
    SELECT 
        g.genre_id genre_id,
        g.name genre_name,
        COUNT(il.quantity) sales_per_genre
    FROM genre g 
    INNER JOIN track t ON t.genre_id = g.genre_id
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = "USA"
    GROUP BY 1
    ),
    total_inv AS
    (
    SELECT 
    c.country country,
    COUNT(il.quantity) total_quantity
    FROM invoice_line il 
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY 1
    )
    
SELECT 
    ciusa.genre_name genre_name,
    ciusa.sales_per_genre,
    CAST(ciusa.sales_per_genre AS FLOAT)/(
                                          SELECT ti.total_quantity
                                          FROM total_inv ti
                                          WHERE ti.country = "USA"
                                          ) percent
FROM count_inv_USA ciusa
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 5
    
/* 
Considering that our online store plays more than one 
genres in music, expanding our catalogue in terms of 
trends is Country specific (and probably city specific).
On a first approximation we find the most popular 
kind of music in the USA, this will give us a good 
direction on which areas we should invest.
A second good approximation would have been to analyze the 
trends in terms of seasoning.

*/

 * sqlite:///chinook.db
Done.


genre_name,sales_per_genre,percent
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423


In [ ]:
%%sql
WITH count_other AS
    (
    SELECT 
        c.customer_id customer_id,
        il.*,
        CASE
            WHEN
            (
            SELECT COUNT(*) FROM customer 
            WHERE country = c.country
            ) =1 THEN "OTHER"
            ELSE c.country
        END AS country
    FROM customer c 
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    INNER JOIN invoice_line il ON i.invoice_id  = il.invoice_id
    )
    
SELECT
    country,
    customers,
    total_sales,
    average_sales_per_customer,
    average_quantity_per_invoice
FROM (
      SELECT 
           country,
           COUNT(DISTINcT(customer_id)) customers,
           SUM(unit_price) total_sales,
           CAST(SUM(unit_price) AS FLOAT)/ COUNT(DISTINCT(customer_id)) average_sales_per_customer,  
           CAST(SUM(unit_price) AS FLOAT)/COUNT(DISTINCT(invoice_id)) average_quantity_per_invoice,
           CASE 
                WHEN country = "OTHER" THEN 1
                ELSE 0
            END AS sort
        FROM count_other 
        GROUP BY 1
        ORDER BY sort ASC, total_sales DESC
      )
/*
Almost obvious to analyze the total sales and number of customers per country. At the same 
time we calculate the units sold per customer and invoice for each country. Chinook 
contains countries with one customer only, we would like to neglect these examples 
from the whole analysis, hence we built a tuple "other" containing all of these
instances.
/*




 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_sales_per_customer,average_quantity_per_invoice
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
OTHER,15,1094.9400000000085,72.99600000000056,7.448571428571486
